In [1]:
import random
import numpy as np
import pandas as pd

random.seed(2)  # Set the seed for Python's random module
np.random.seed(3)  # Set the seed for numpy's random module

In [2]:
from cabm import cabm_model

In [3]:
model = cabm_model.ConsumerModel(1000, "config_ALL_ON.toml", enable_ads=True, enable_pricepoint=True, enable_ad_increment=True)

/Users/MikeFitzgerald/Documents/GitHub/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/Users/MikeFitzgerald/Documents/GitHub/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/Users/MikeFitzgerald/venvs/py311_mesa/lib/python3.11/site-packages/mesa/time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If

In [4]:
num_steps = 104

for i in range(num_steps):
    model.step()

In [5]:
agent_df = model.datacollector.get_agent_vars_dataframe()

In [6]:
#agent_df.to_pickle('agent_output_NIL.pkl')

In [7]:
agent_df

Household_Size  Consumption_Rate Brand_Preference Brand_Choice  \
Step AgentID                                                                   
0    0                     2          1.970529                A            A   
     1                     3          3.884622                A            A   
     2                     2          3.156573                B            B   
     3                     3          3.845953                A            A   
     4                     2          2.010388                A            A   
...                      ...               ...              ...          ...   
103  907                   3          1.700390                A            A   
     943                   2          1.116864                A            A   
     756                   2          3.474394                B            B   
     960                   3          3.536195                B            B   
     838                   1          3.424736                B            B   

              Loyalty_Rate                             Purchase_Probabilities  \
Step AgentID                                                                    
0    0            0.955641  {'A': 0.9556411825293621, 'B': 0.0443588174706...   
     1            0.943237  {'A': 0.9432372283159218, 'B': 0.0567627716840...   
     2            0.932727  {'A': 0.0672734411797149, 'B': 0.9327265588202...   
     3            0.994624  {'A': 0.9946242956713564, 'B': 0.0053757043286...   
     4            0.899391  {'A': 0.8993911785732669, 'B': 0.1006088214267...   
...                    ...                                                ...   
103  907          0.726036  {'A': 0.6065393948496819, 'B': 0.3934606051503...   
     943          0.928056  {'A': 0.9329994490749014, 'B': 0.0670005509250...   
     756          0.932762  {'A': 0.04688312004888231, 'B': 0.953116879951...   
     960          0.921355  {'A': 0.0575879226269891, 'B': 0.9424120773730...   
     838          0.906239  {'A': 0.06559419139036086, 'B': 0.934405808609...   

              Enable_Ads  Ad_Decay_Factor  \
Step AgentID                                
0    0              True         1.942079   
     1              True         2.982367   
     2              True         1.231164   
     3              True         3.123978   
     4              True         1.732238   
...                  ...              ...   
103  907            True         2.193041   
     943            True         1.013978   
     756            True         3.291853   
     960            True         1.898931   
     838            True         2.790448   

                                          Ad_Channel_Preference  \
Step AgentID                                                      
0    0        {'TV': 0.5726599461566402, 'Web': 0.4273400538...   
     1        {'TV': 0.14834217656671572, 'Web': 0.851657823...   
     2        {'TV': 0.38906340654938737, 'Web': 0.610936593...   
     3        {'TV': 0.26547286013568405, 'Web': 0.734527139...   
     4        {'TV': 0.36873839722004226, 'Web': 0.631261602...   
...                                                         ...   
103  907      {'TV': 0.3877125835620585, 'Web': 0.6122874164...   
     943      {'TV': 0.24099577011093318, 'Web': 0.759004229...   
     756      {'TV': 0.2612686013812564, 'Web': 0.7387313986...   
     960      {'TV': 0.457601857018594, 'Web': 0.54239814298...   
     838      {'TV': 0.39010162277180477, 'Web': 0.609898377...   

                                                       Adstock  ...  \
Step AgentID                                                    ...   
0    0                                        {'A': 0, 'B': 0}  ...   
     1                                        {'A': 0, 'B': 0}  ...   
     2                                        {'A': 0, 'B': 0}  ...   
     3                                        {'A': 0, 'B': 0}  ...   
     4                             

In [10]:
agent_df.columns

Index(['Household_Size', 'Consumption_Rate', 'Brand_Preference',
       'Brand_Choice', 'Loyalty_Rate', 'Purchase_Probabilities', 'Enable_Ads',
       'Ad_Decay_Factor', 'Ad_Channel_Preference', 'Adstock', 'Pantry_Min',
       'Pantry_Max', 'Pantry_Stock', 'Purchased_This_Step', 'Current_Price',
       'Last_Product_Price', 'Purchase_Behavior', 'Step_Min', 'Step_Max',
       'price_A', 'tv_A', 'web_A', 'price_B', 'tv_B', 'web_B'],
      dtype='object')

In [11]:
agent_df.index

MultiIndex([(  0,   0),
            (  0,   1),
            (  0,   2),
            (  0,   3),
            (  0,   4),
            (  0,   5),
            (  0,   6),
            (  0,   7),
            (  0,   8),
            (  0,   9),
            ...
            (207, 132),
            (207, 348),
            (207, 411),
            (207, 650),
            (207, 140),
            (207, 679),
            (207, 446),
            (207, 165),
            (207, 829),
            (207, 730)],
           names=['Step', 'AgentID'], length=208000)